# Dennis Pierantozzi & Francesco Maria Mosca

## Practice 1 CNNs

### Loading the dataset

In [4]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Define the relative paths to your train and test directories
train_dir = os.path.join(current_dir, 'animals', 'train')
test_dir = os.path.join(current_dir, 'animals', 'val')

### Preprocessing the data

What have we done here? <br>
We have used the function the professor suggested and preprocessed the data <br>
for the train data the labels are inferred by the directory and categorical encoded. Size fixed in 120 and 120 and shuffled <br>
for the test same but not shuffled since it is not needed <br>
#### Created a validation dataset from the training dataset (split: 0.2)

In [5]:
from keras.utils import image_dataset_from_directory

# Create TensorFlow datasets for training and validation
train_data = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32,  # Batch size
    shuffle=True,  # Shuffle the data
    seed=123,  # Random seed for shuffling
    validation_split=0.2,  # Fraction of training data to use for validation
    subset='training'  # Use the training subset
)

validation_data = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32,  # Batch size
    shuffle=False,  # Do not shuffle the data
    seed=123,  # Random seed for shuffling
    validation_split=0.2,  # Fraction of training data to use for validation
    subset='validation'  # Use the validation subset
)

test_data = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',  # For one-hot encoded labels
    image_size=(120, 120),  # Resize images to (120, 120)
    batch_size=32  # Batch size
)

Found 13474 files belonging to 5 classes.
Using 10780 files for training.
Found 13474 files belonging to 5 classes.
Using 2694 files for validation.
Found 1497 files belonging to 5 classes.


### Data agumentation

Why data agumentation? Which kind of things are we doing here? 

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.Rescaling(1./255)
    ]
)